# Group Assignment QF627

*Group Members:*
* Anna Germaine Lim
* Peng Cheng
* Zenith Tay
* Gregory Tan

# Packages Used in This Workbook

In [261]:
## Data Download

import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

## For Visualisation

# Useful Functions Used in this Sheet

In [265]:
def download_data (ticker,
                   start_date: str | datetime,
                   end_date: str | datetime) -> pd.DataFrame:
    data =\
    (
        yf.download(tickers = ticker,
                    start = start_date,
                    end = end_date)
    )

    return data

In [266]:
# preallocate empty array and assign slice by chrisaycock

def np_shift(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

### Mean Reversion Strategies

In [267]:
def bollinger_band(price_data: pd.Series,
                            window: int = 14,
                                                ) -> pd.Series:

    price = price_data[price_col]

    std_dev_series =\
    (
        price
        .rolling(window = window)
        .std()
    )

    price_high =\
    (
        price + 2*std_dev_series
    )

    price_low =\
    (
        price - 2*std_dev_series
    )

    return price_high, price_low

### Momentum

In [268]:
def generate_moving_avg(price_data: pd.Series,
                        window: int
                                              ) -> pd.Series:
    
    ma_series =\
    (
        pd.Series
        (   
            price_data
            .rolling(window = window)
            .mean(),

            name = 'MA' + str(window)
        )
    )

    return ma_series

In [269]:
# Exponential Moving Average

def EMA(price_data: pd.Series, 
        window: int
                            ) -> pd.Series:
    EMA = pd.Series(price_data
                    .ewm(span = window,
                         min_periods = window)
                    .mean(),
                    name = "EMA_" + str(window)
                    )
    return EMA

In [ ]:
def generate_moving_avg_cross_signal(long_ma: pd.Series,
                                    short_ma: pd.Series) -> pd.Series:
    
    ## Sanity Check
    if len(long_ma) != len(short_ma):
        print('MA series lengths not equal, please check')
        return

    ## Return Signals
    else:

        moving_avg_cross_positions = np.where(short_ma > long_ma, 1.0, 0.0)
        moving_avg_cross_positions = np.where(short_ma < long_ma, -1.0 , moving_avg_cross_positions)

        moving_avg_cross_signals = np.where(moving_avg_cross_positions - np_shift(moving_avg_cross_positions,1) > 0, 1, 0)
        moving_avg_cross_signals = np.where(moving_avg_cross_positions - np_shift(moving_avg_cross_positions, 1) < 0, -1, moving_avg_cross_signals)

        buy_or_sell = pd.DataFrame({'MA_Cross_Signal':moving_avg_cross_signals, 'MA_Cross_Position': moving_avg_cross_positions},
                                   index = long_ma.index
                                   )

        return buy_or_sell

In [355]:
def generate_rate_of_change(price_data: pd.Series,
                            n: int
                            ) -> pd.Series:
    
    ROC = pd.Series(
                        (price_data - price_data.diff(n)) / price_data.diff(n),
                        name = 'ROC'+str(n),
                        # index = price_data.index
                    )
    
    return ROC

In [457]:
def generate_rate_of_change_signal(roc_data: pd.Series) -> pd.Series:

    roc_position = pd.Series(np.where(roc_data > 0, 1.0, 0.0), index=roc_data.index, name = 'ROC_Position')
    roc_signal = roc_position.diff()
    roc_signal.name = 'ROC_Signal'
    
    # roc_signal = roc_position - np_shift(roc_position, 1)


    # buy_or_sell = pd.DataFrame({'ROC_Position': roc_position, 'ROC_Signal': roc_signal},
    #                         #    index = roc_data.index
    #                            )

    return pd.concat([roc_position, roc_signal], axis=1)

In [ ]:
def annual_sharpe(returns):
    days = (returns.index[-1] - returns.index[0]).days
    
    return\
    (
        (
            (1+returns).prod()
            **(365/days) 
            - 1
        )
        /
        returns.std()
        /
        np.sqrt(252)
    )

In [273]:
def RSI(series, period):
    
    delta = series.diff().dropna()
    
    u = delta * 0
    d = u.copy()
    
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    
    u[u.index[period - 1]] = np.mean( u[:period] ) # 
    
    u = u.drop(u.index[:(period - 1)
                      ]
              )
    
    d[d.index[period - 1]] = np.mean( d[:period] )
    
    d = d.drop(d.index[:(period - 1)
                      ]
              )
    
    rs = u.ewm(com = period - 1, adjust = False).mean() / \
         d.ewm(com = period - 1, adjust = False).mean()
    
    return 100 - 100 / (1 + rs)

## Performance Metrics

In [ ]:
def annual_sharpe(returns):
    days = (returns.index[-1] - returns.index[0]).days
    
    return\
    (
        (
            (1+returns).prod()
            **(365/days) 
            - 1
        )
        /
        returns.std()
        /
        np.sqrt(252)
    )

In [ ]:
## CAGR

def cagr(returns: pd.Series) -> float:
    days = (returns.index[-1] - returns.index[0]).days
    return ( (1 + returns).prod() )**(365/days) - 1   

In [335]:
### Max Drawdown

def max_drawdown(cumulative_returns):
    max_performance = cumulative_returns.cummax()
    dd = ((max_performance - cumulative_returns) / max_performance).max()
    return dd


### Longest Drawdown

def calculate_longest_drawdown(cumulative_returns):
    drawdown = cumulative_returns.cummax() - cumulative_returns
    period =\
    (
        np
        .diff(np
              .append(drawdown[drawdown == 0].index, 
                      drawdown.index[-1: ]
                    )
            )
    )
    return period.max() / np.timedelta64(1, "D")

In [483]:
def evaluate_returns(returns_series: pd.Series):
    
    cum_returns_series = (1 + returns_series).cumprod()

    tot_returns = (1 + returns_series).prod()
    CAGR = cagr(returns_series)
    Annualised_Sharpe = annual_sharpe(returns_series)
    Max_DD = max_drawdown(cum_returns_series)
    Longest_DD = calculate_longest_drawdown(cum_returns_series)


    print('-- Summary of Returns -- \n',
          f'Total Returns: {tot_returns: .2%} \n',
          f'CAGR: {CAGR: .2%} \n',
          f'Annualised_Sharpe: {Annualised_Sharpe: .2%} \n',
          f'Max Drawdown: {Max_DD: .2%} \n',
          f'Longest Drawdown (Days): {Longest_DD}'            
          )

    return pd.Series([tot_returns, CAGR, Annualised_Sharpe, Max_DD, Longest_DD])

## Packages

In [277]:
## Data Download

import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

### Visualisation

## Download Dataset

In [493]:
train_proportion = 0.75

spy_data =\
(
    download_data('SPY',
                  start_date = '2006-11-01',
                  end_date = '2025-11-12')
    .droplevel(level = 1,
               axis = 1)
)

spy_train_data = spy_data[:int(train*len(spy_data))]
spy_test_data = spy_data[int(train*len(spy_data)):]


spy_data_close = spy_train_data['Close'].to_frame()
spy_data_returns = spy_train_data['Close'].pct_change().to_frame().rename(columns= {'Close': 'Returns'})

C:\Users\TanFamily4\AppData\Local\Temp\ipykernel_16024\21668489.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf.download(tickers = ticker,
[*********************100%***********************]  1 of 1 completed


In [ ]:
spy_data_returns

,Returns
Date,
2006-11-01,NaN
2006-11-02,-0.000585
2006-11-03,-0.001755
2006-11-06,0.011279
2006-11-07,0.003838
...,...
2021-02-01,0.016645
2021-02-02,0.014140
2021-02-03,0.000786


In [501]:
evaluate_returns(spy_data_returns['Returns'])

-- Summary of Returns -- 
 Total Returns:  378.22% 
 CAGR:  9.77% 
 Annualised_Sharpe:  47.66% 
 Max Drawdown:  55.19% 
 Longest Drawdown (Days): 1773.0


0       3.782161
1       0.097678
2       0.476599
3       0.551894
4    1773.000000
dtype: float64

## Momentum Strategies

In [495]:
time_periods = np.arange(10, 201, 10).tolist()

#### ROC

In [496]:
pd.Series.name

In [ ]:
ROC_Metrics = pd.DataFrame()

for i in time_periods:
    roc_data = spy_data_returns.copy()
    
    roc_data[f'ROC{i}'] = generate_rate_of_change(spy_data_close['Close'], i)
    roc_data = pd.concat([roc_data, generate_rate_of_change_signal(roc_data[f'ROC{i}'])], axis = 1)
    roc_data['Strat_returns'] = roc_data['Returns'] * roc_data['ROC_Position'].shift(1)
    
    print(f'===Data for ROC{i}===')
    roc_series = evaluate_returns(roc_data['Strat_returns'])
    roc_series.name = f'ROC{i}'
    
    ROC_Metrics = pd.concat([ROC_Metrics, roc_series], axis = 1)

ROC_Metrics.index = ['Total Returns', 'CAGR', 'Annualised Sharpe', 'Max Drawdown', 'Longest Drawdown (Days)']
ROC_Metrics.T

===Data for ROC10===
-- Summary of Returns -- 
 Total Returns:  168.31% 
 CAGR:  3.71% 
 Annualised_Sharpe:  31.32% 
 Max Drawdown:  45.35% 
 Longest Drawdown (Days): 4178.0
===Data for ROC20===
-- Summary of Returns -- 
 Total Returns:  248.25% 
 CAGR:  6.58% 
 Annualised_Sharpe:  59.26% 
 Max Drawdown:  26.66% 
 Longest Drawdown (Days): 864.0
===Data for ROC30===
-- Summary of Returns -- 
 Total Returns:  222.97% 
 CAGR:  5.78% 
 Annualised_Sharpe:  51.64% 
 Max Drawdown:  28.71% 
 Longest Drawdown (Days): 1318.0
===Data for ROC40===
-- Summary of Returns -- 
 Total Returns:  268.57% 
 CAGR:  7.17% 
 Annualised_Sharpe:  62.99% 
 Max Drawdown:  33.84% 
 Longest Drawdown (Days): 1205.0
===Data for ROC50===
-- Summary of Returns -- 
 Total Returns:  230.08% 
 CAGR:  6.01% 
 Annualised_Sharpe:  53.33% 
 Max Drawdown:  24.29% 
 Longest Drawdown (Days): 1023.0
===Data for ROC60===
-- Summary of Returns -- 
 Total Returns:  289.40% 
 CAGR:  7.73% 
 Annualised_Sharpe:  67.89% 
 Max Drawdown:

,Total Returns,CAGR,Annualised Sharpe,Max Drawdown,Longest Drawdown (Days)
ROC10,1.683071,0.037147,0.313249,0.453508,4178.0
ROC20,2.482465,0.065773,0.592612,0.266626,864.0
ROC30,2.229748,0.057786,0.516353,0.287065,1318.0
ROC40,2.685687,0.071664,0.629924,0.338358,1205.0
ROC50,2.300767,0.060112,0.533288,0.242892,1023.0
ROC60,2.894019,0.077288,0.678907,0.163700,860.0
ROC70,1.830952,0.043284,0.376869,0.316276,1300.0
ROC80,2.144994,0.054919,0.484968,0.283964,1106.0
ROC90,2.066280,0.052159,0.457029,0.269074,936.0
ROC100,2.353124,0.061785,0.524384,0.237623,879.0


In [ ]:
for i in time_periods:
    roc_data

Date
2006-11-01           NaN
2006-11-02           NaN
2006-11-03           NaN
2006-11-06           NaN
2006-11-07           NaN
                 ...    
2025-11-05     68.282003
2025-11-06   -463.278865
2025-11-07   -107.841854
2025-11-10   -180.326892
2025-11-11   -169.226702
Name: ROC10, Length: 4787, dtype: float64